In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
import pandas as pd

# Task 4: Агрегации

На предыдущих этапах вы занимались просто подготовкой данных. Вы не получили никакой аналитической информации на основе имеющихся данных. Пришло время приступить к анализу. Помните, что, согласно нашему предположению, пользователи приступили бы к работе над лабораторными заданиями раньше, если бы они просмотрели ленту новостей? Это означает, что ключевой метрикой для нас является период времени (delta) между датой начала работы пользователя над лабораторным заданием (первого коммита) и сроком сдачи лабораторного задания. Мы будем смотреть на то, меняется ли она в зависимости от просмотра страницы.

Что вам нужно сделать в этом упражнении:

## 1. Создайте соединение с базой данных с помощью библиотеки sqlite3

In [34]:
import sqlite3
con = sqlite3.connect('/content/drive/MyDrive/School21/day11/data/checking-logs.sqlite')

## 2. Получите схему таблицы test

In [35]:
pd.io.sql.read_sql("PRAGMA table_info(test)", con)

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


## 3. Получите только первые 10 строк таблицы test, чтобы проверить, как она выглядит

In [36]:
pd.io.sql.read_sql("SELECT * from test LIMIT 10", con)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


## 4. Найдите среди всех пользователей минимальное значение этой самой дельты (периода времени от даты первого коммита пользователя до срока сдачи соответствующего лабораторного задания), используя только один запрос.

- Для этого сджойните данные своей таблицы с данными таблицы deadlines

- Период времени должен отображаться в часах.
- Не учитывайте лабораторное задание project1 с более длительным сроком сдачи, поскольку оно будет выделяться на фоне других.
- Сохраните значение в датафрейме df_min с соответствующим uid.

- [strftime функция SQLite](https://oracleplsql.ru/strftime-sqlite.html)

In [37]:
pd.io.sql.read_sql("PRAGMA table_info(deadlines)", con)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,labs,TEXT,0,None,0
2,2,deadlines,INTEGER,0,None,0


In [38]:
pd.io.sql.read_sql("SELECT * from deadlines LIMIT 3", con)

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399


In [39]:
df_min = pd.io.sql.read_sql("""
          SELECT test.uid, min(((strftime('%s', test.first_commit_ts) - deadlines.deadlines) /3600)) as delta_min
          FROM test
          LEFT JOIN deadlines
          ON test.labname = deadlines.labs
          WHERE test.labname != 'project1'
""", con)

In [40]:
df_min

,uid,delta_min
0,user_30,-202


## 5. Выполните те же самые операции для максимального значения дельты, используя только один запрос. Название итогового датафрейма — df_max

In [41]:
df_max = pd.io.sql.read_sql("""
          SELECT test.uid, max(((strftime('%s', test.first_commit_ts) - deadlines.deadlines) /3600)) as delta_max
          FROM test
          LEFT JOIN deadlines
          ON test.labname = deadlines.labs
          WHERE test.labname != 'project1'
""", con)

In [42]:
df_max

,uid,delta_max
0,user_25,-2


## 6. Выполните те же самые операции для среднего значения дельты, используя только один запрос. На этот раз ваш итоговый датафрейм не должен включать столбец uid; имя датафрейма — df_avg

In [43]:
df_avg = pd.io.sql.read_sql("""
          SELECT avg(((strftime('%s', test.first_commit_ts) - deadlines.deadlines) /3600)) as delta_avg
          FROM test
          LEFT JOIN deadlines
          ON test.labname = deadlines.labs
          WHERE test.labname != 'project1'
""", con)

In [44]:
df_avg

,delta_avg
0,-89.125


## 7. Мы хотим проверить предположение о том, что у пользователей, посетивших ленту новостей всего несколько раз, меньше период времени между датой первого коммита и сроком сдачи лабораторного задания. Для этого вам необходимо рассчитать коэффициент корреляции между количеством просмотров страниц и разницей между датами.


- Используя только один запрос, создайте таблицу со столбцами: uid, avg_diff, pageviews.

  * uid — это uid, существующие в таблице test.

  * avg_diff — среднее значение дельты (периода времени между датой первого коммита и сроком сдачи лабораторного задания) для каждого пользователя.

  * pageviews — количество посещений ленты новостей одним пользователем.


- Не учитывайте лабораторное задание project1.
- Сохраните результаты в датафрейме views_diff.
- Используйте метод corr() библиотеки Pandas для вычисления коэффициента корреляции между количеством просмотров и значением дельты.

In [47]:
correlation_coefficient = """

CREATE TABLE views_diff AS
  SELECT test.uid, avg(delta) as avg_diff , pageviews
  FROM 
    (SELECT test.uid, ((strftime('%s', test.first_commit_ts) - deadlines.deadlines) /3600) as delta
    FROM test
    LEFT JOIN deadlines
    ON test.labname = deadlines.labs
    WHERE test.labname != 'project1') as test

  LEFT JOIN
    (SELECT uid, count(datetime) as pageviews
    FROM pageviews
    group by uid) as pageviews

  ON test.uid = pageviews.uid
  GROUP BY test.uid

"""

con.cursor().execute(correlation_coefficient)
con.commit()

In [48]:
#pd.read_sql_query('DROP TABLE views_diff', con)

In [49]:
views_diff = pd.io.sql.read_sql("SELECT * from views_diff", con)

In [50]:
views_diff.corr()

,avg_diff,pageviews
avg_diff,1.000000,-0.279736
pageviews,-0.279736,1.000000


## 8. Закройте соединение

In [51]:
con.close()